In [2]:
!pip install -q einops==0.7.0 langchain==0.1.9 pypdf==4.0.2 sentence-transformers==2.4.0
!pip install mysql-connector-python

import os
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import mysql.connector

# Caminho para a pasta com os PDFs
pdf_folder_path = "estatutos"

# Carrega os PDFs da pasta
pdf_loader = PyPDFDirectoryLoader(pdf_folder_path)
pdf_docs = pdf_loader.load()
docs = pdf_docs

# Divide os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
all_splits = text_splitter.split_documents(docs)

# Configura a conexão com o MariaDB
db_connection = mysql.connector.connect(
    host="mariadb.banco-galicia.svc.cluster.local",
    user="",
    password="",
    database=""
)

cursor = db_connection.cursor()

# Cria a tabela para armazenar os documentos
create_table_query = """
CREATE TABLE IF NOT EXISTS pdf_documents (
    id INT AUTO_INCREMENT PRIMARY KEY,
    source VARCHAR(255),
    content TEXT
)
"""

cursor.execute(create_table_query)
db_connection.commit()

# Insere os documentos na tabela
for doc in all_splits:
    source = doc.metadata.get('source', '')
    content = doc.page_content
    insert_query = "INSERT INTO pdf_documents (source, content) VALUES (%s, %s)"
    cursor.execute(insert_query, (source, content))

db_connection.commit()

cursor.close()
db_connection.close()



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
